Evaluation result:

54 Yes

46 No

In [ ]:
!pip install transformers
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/gdrive/MyDrive/CSI6900/pre_processing.py /content

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.8 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import random
from tqdm.auto import tqdm
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Run locally
#DATA_PATH = ''

# colab
DATA_PATH = '/content/gdrive/MyDrive/CSI6900/'

device = torch.device("mps" if getattr(torch,'has_mps',False) else "cuda:0" if torch.cuda.is_available() else "cpu")
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


device(type='cuda', index=0)

## Load data

In [ ]:
infoSheet = pd.read_csv(DATA_PATH + 'data/infoSheets_2023-05-18.csv')
# Drop rows where 'taxonomy heading ids' is '0'
infoSheet.drop(infoSheet[infoSheet['taxonomy heading ids'] == '0'].index, inplace = True)
infoSheet.reset_index(drop=True, inplace=True)
print(infoSheet.isnull().sum())
print('Length: ', len(infoSheet.index))
infoSheet.head(10)

ID                       0
name_en                  0
name_fr                 91
abstract_en              7
abstract_fr             96
description_en           0
description_fr          91
taxonomy heading ids     0
dtype: int64
Length:  237


,ID,name_en,name_fr,abstract_en,abstract_fr,description_en,description_fr,taxonomy heading ids
0,8920,Abuse and Domestic Violence,Maltraitance et violence familiale,"Abuse is behaviour used to intimidate, isolate...",La maltraitance est un comportement visant à i...,\r\n\tWhat is Abuse and Domestic Violence?\r\n...,\r\n\tQu&#39;est-ce que la maltraitance et la ...,"0,219,58,8,7,6,50,9,36,54,37"
1,69660,"ADHD in Children, Youth and Adults: Informatio...",NaN,Attention deficit hyperactivity disorder (ADHD...,NaN,"\r\n\tAbbreviations\r\n\r\n\tADHD, attention-d...",NaN,"0,13"
2,20505,ADHD: Managing ADHD Medications,TDAH: Gestion des médicaments pour le TDAH,Although ADHD medications may be helpful for s...,Bien que les médicaments du TDAH peuvent être ...,\r\n\tGeneral Considerations\r\n\r\n\t\r\n\t\t...,\r\n\tObservations générales\r\n\r\n\t\r\n\t\t...,"0,13"
3,8917,Alcohol and Substance Use Problems in Children...,Problèmes de l’alcoolisme et la toxicomanie ch...,Many children/youth experiment with alcohol an...,Nombreux sont les enfants et les jeunes qui fo...,\r\n\tDavid's Story\r\n\r\n\tUp until this sch...,\r\n\tL’histoire de David\r\n\r\n\tJusqu’à cet...,"0,12"
4,8909,Alcohol: Cutting Back or Quitting Drinking,Problèmes liés à la consommation d'alcool,Drinking alcohol is an accepted practice in ma...,La consommation d'alcool est une pratique acce...,\r\n\tIntroduction\r\n\r\n\tMany people who dr...,\r\n\tIntroduction\r\n\r\n\tNombreuses sont le...,"0,12"
5,61003,Alcohol Use Disorder in Adults: Information fo...,NaN,Alcohol use problems are common in primary car...,NaN,\r\n\tEpidemiology\r\n\r\n\tPrevalence\r\n\r\n...,NaN,"0,12,266"
6,18393,Alcohol Use Disorder in Youth: Information for...,NaN,Many youth will experiment with alcohol and su...,NaN,Epidemiology\r\n\r\n\t\r\n\t\t10% of the&nbsp;...,NaN,"0,12,266"
7,24390,Alzheimer's Dementia: Information for Primary ...,NaN,Alzheimer’s is the most common form of dementi...,NaN,\r\n\tCase\r\n\r\n\t\r\n\t\tYour 80 year old p...,NaN,"0,79"
8,11564,Anger,La colère,Anger is a normal emotion that we get when we ...,La colère est une émotion normale que nous res...,\r\n\tDoes any of this sound like you?\r\n\r\n...,\r\n\tEst-ce que cela trouve un &eacute;cho ch...,"0,14,50,54,9,36,37"
9,26975,Anorexia Nervosa: Information for Primary Care,NaN,Anorexia nervosa is a common eating disorder a...,NaN,\r\n\tCase\r\nIdentifying data&nbsp;32-yo fema...,NaN,"0,22"


In [ ]:
taxonomy = pd.read_json(DATA_PATH + 'data/taxonomy_headings.json')
taxonomy = taxonomy.drop(['created_at',	'updated_at',	'deleted_at', 'alias_of_id', 'short_description',	'original_id'], axis=1)
print(taxonomy.isnull().sum())
print('Length: ', len(taxonomy.index))
taxonomy.head(10)

id              0
name            0
description     0
translations    0
dtype: int64
Length:  237


,id,name,description,translations
0,1,Root,Root,"{""name"":{""en"":""Root"",""fr"":null},""description"":..."
1,2,All Mental Health Resources,<p>\r\n\tThe listings of mental health resourc...,"{""name"":{""en"":""All Mental Health Resources"",""f..."
2,3,Crisis and Emergency,<p>\r\n\tRefers to all programs that provide i...,"{""name"":{""en"":""Crisis and Emergency"",""fr"":""Res..."
3,4,"System Navigation, including Information and R...","<p>\r\n\tAre you looking for help, but don&#39...","{""name"":{""en"":""System Navigation, including In..."
4,5,Child Welfare including Children's Aid Society...,<p>The child welfare / child protection system...,"{""name"":{""en"":""Child Welfare including Childre..."
5,6,Emergency Shelter and Housing,<p>\r\n\tThere are various shelters that peopl...,"{""name"":{""en"":""Emergency Shelter and Housing"",..."
6,7,Hospital Emergency Department,<p>\r\n\tIs there an emergency such as medical...,"{""name"":{""en"":""Hospital Emergency Department"",..."
7,8,"Crisis Lines including Telephone, Online and Chat",<p>\r\n\tAre you in a crisis? Crisis lines off...,"{""name"":{""en"":""Crisis Lines including Telephon..."
8,9,Psychiatrists,<p>\r\n\tPsychiatrists are medical doctors who...,"{""name"":{""en"":""Psychiatrists"",""fr"":""Psychiatre..."
9,10,A-Z Mental Health Conditions and Topics,<p>\r\n\tAlphabetical list of mental health to...,"{""name"":{""en"":""A-Z Mental Health Conditions an..."


## Pre-processing

In [ ]:
from pre_processing import remove_empty, remove_HTML, remove_new_line

infoSheet['description_en'] = infoSheet['description_en'].apply(lambda x: remove_HTML(x))
infoSheet['description_en'] = infoSheet['description_en'].apply(lambda x: remove_new_line(x))

print('Length of taxonomy before preprocessing:', len(taxonomy.index))
taxonomy['description'] = taxonomy['description'].apply(lambda x: remove_HTML(x))
taxonomy['description'] = taxonomy['description'].apply(lambda x: remove_new_line(x))
# taxonomy = remove_empty('description', taxonomy)
print('Length of taxonomy after preprocessing:', len(taxonomy.index))

Length of taxonomy before preprocessing: 277
Length of taxonomy after preprocessing: 277


## Prediction

In [ ]:
# Get models - The package will take care of downloading the models automatically
# For best performance: EleutherAI/gpt-j-6B
# EleutherAI/gpt-neo-1.3B
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").to(device)
# Deactivate Dropout (There is no dropout in the above models so it makes no difference here but other SGPT models may have dropout)
model.eval()

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj):

In [ ]:
import spacy

# keyword search (optional)
def keyword_search(query, docs):
    nlp = spacy.load('en_core_web_sm')
    stop_words = nlp.Defaults.stop_words

    result_docs = []
    # tokenize query and remove stop words
    query_tokens = word_tokenize(query.lower())
    query_tokens = [w for w in query_tokens if not w in stop_words]
    for doc in docs:
        # tokenize doc and remove stop words
        doc_tokens = word_tokenize(doc.lower())
        doc_tokens = [w for w in doc_tokens if not w in stop_words]

        # check if tokens in query appeared at least once in doc
        for each in query_tokens:
            if each in doc_tokens:
                result_docs.append(doc)
                break

    print('Original %d, found %d' % (len(docs), len(result_docs)))
    return result_docs

In [ ]:
def search(queries, docs, keywordSearch, topK=10):
    prompt = 'Documents are searched to find matches with the relevant content.\nThe document "{}" is a good search result for "'

    if not isinstance(queries, list):
        queries = [queries]
    if not isinstance(docs, list):
        docs = [docs]
    if topK > len(docs):
        topK = len(docs)

    result = []
    for queryIdx, query in enumerate(tqdm(queries, desc="Searching", colour="#00ff00")):
    # for query in queries:
        tmp = []

        if keywordSearch:
            tmp_docs = keyword_search(query, docs)
        else:
            tmp_docs = docs

        for _, doc in enumerate(tqdm(tmp_docs, desc=f"Query {queryIdx + 1}/{len(queries)}", colour="#00ff00")):
        # for doc in docs:
            context = prompt.format(doc)

            context_enc = tokenizer.encode(context, add_special_tokens=False)
            continuation_enc = tokenizer.encode(query, add_special_tokens=False)
            # Slice off the last token, as we take its probability from the one before
            model_input = torch.tensor(context_enc+continuation_enc[:-1]).to(device)
            continuation_len = len(continuation_enc)
            input_len, = model_input.shape

            # [seq_len] -> [seq_len, vocab]
            logprobs = torch.nn.functional.log_softmax(model(model_input)[0], dim=-1).cpu()
            # [seq_len, vocab] -> [continuation_len, vocab]
            logprobs = logprobs[input_len-continuation_len:]
            # Gather the log probabilities of the continuation tokens -> [continuation_len]
            logprobs = torch.gather(logprobs, 1, torch.tensor(continuation_enc).unsqueeze(-1)).squeeze(-1)
            score = torch.sum(logprobs).item()
            # The higher (closer to 0), the more similar
            tmp.append(score)
        top_results = torch.topk(torch.tensor(tmp), topK)
        result.append(top_results)

    return result

In [ ]:
# Randomly choose 10 infoSheets
search_term_indices = [random.randint(0, len(infoSheet.index)) for i in range(10)]
search_term_indices

[71, 88, 167, 4, 10, 103, 175, 206, 113, 176]

In [ ]:
# Prepare data
infoSheet_data = []
for each in search_term_indices:
    if infoSheet['taxonomy heading ids'][each] != '0':
        infoSheet_data.append(infoSheet['name_en'][each] + ': ' + infoSheet['abstract_en'][each])

In [ ]:
taxonomy_description = taxonomy['description'].tolist()
taxonomy_name = taxonomy['name'].tolist()

taxonomy_data = []
for i in range(len(taxonomy_name)):
    taxonomy_data.append(taxonomy_name[i] + ': ' + taxonomy_description[i])

In [ ]:
result = search(infoSheet_data, taxonomy_data, True)

predictions = {'infoSheet_index': [], 'infoSheet_name': [], 'pred_taxonomy_index': [], 'pred_taxonomy_name': [], 'score': [], 'gold_taxonomy_index': []}
for q in range(len(result)):
    for score, idx in zip(result[q][0], result[q][1]):
        predictions['infoSheet_index'].append(search_term_indices[q])
        predictions['infoSheet_name'].append(infoSheet['name_en'][search_term_indices[q]])
        predictions['pred_taxonomy_index'].append(idx.item()+1)
        predictions['pred_taxonomy_name'].append(taxonomy['name'][idx.item()])
        predictions['score'].append(score.item())
        predictions['gold_taxonomy_index'].append(infoSheet['taxonomy heading ids'][search_term_indices[q]].split(','))

predictions = pd.DataFrame(predictions)
predictions.head(10)

Searching:   0%|          | 0/10 [00:00<?, ?it/s]

Original 277, found 277


Query 1/10:   0%|          | 0/277 [00:00<?, ?it/s]

Original 277, found 277


Query 2/10:   0%|          | 0/277 [00:00<?, ?it/s]

...

Original 277, found 277


Query 9/10:   0%|          | 0/277 [00:00<?, ?it/s]

Original 277, found 277


Query 10/10:   0%|          | 0/277 [00:00<?, ?it/s]

,infoSheet_index,infoSheet_name,pred_taxonomy_index,pred_taxonomy_name,score,gold_taxonomy_index
0,71,Dissociative Identity Disorder (DID),236,Social Anxiety Disorder,-225.853790,"[0, 59, 35, 52]"
1,71,Dissociative Identity Disorder (DID),20,Developmental Coordination Disorder (DCD),-233.090790,"[0, 59, 35, 52]"
2,71,Dissociative Identity Disorder (DID),77,Cognitive Behavioural Therapy (CBT),-236.290833,"[0, 59, 35, 52]"
3,71,Dissociative Identity Disorder (DID),273,Transcranial Magnetic Stimulation (TMS),-236.885834,"[0, 59, 35, 52]"
4,71,Dissociative Identity Disorder (DID),71,Suicide,-237.238358,"[0, 59, 35, 52]"
5,71,Dissociative Identity Disorder (DID),42,Acquired Immune Deficiency Syndrome (AIDS),-238.385086,"[0, 59, 35, 52]"
6,71,Dissociative Identity Disorder (DID),247,Dyadic Developmental Psychotherapy (DDP),-239.715759,"[0, 59, 35, 52]"
7,71,Dissociative Identity Disorder (DID),208,"Antisocial Personality Disorder (ASPD), Psycho...",-239.781860,"[0, 59, 35, 52]"
8,71,Dissociative Identity Disorder (DID),9,Psychiatrists,-240.288162,"[0, 59, 35, 52]"
9,71,Dissociative Identity Disorder (DID),261,Brainspotting (BSP),-240.626953,"[0, 59, 35, 52]"


## Evaluation

In [ ]:
retrieved_relevant = 0
for _, row in predictions.iterrows():
    if str(row['pred_taxonomy_index']) in row['gold_taxonomy_index']:
        retrieved_relevant += 1
print(retrieved_relevant)
print('Precision: ' + str(retrieved_relevant / len(predictions.index)))

4
Precision: 0.04


## Export result

In [ ]:
predictions.to_csv(DATA_PATH + 'data/predictions/sgpt-ce-1.3B.csv', index=False)